In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import sys

from modules.path_planning.astar import AStar
from modules.path_planning.BFS import BFS
from modules.path_planning.greedyBestFirst import greedyBestFirst
from modules.path_planning.bidirectDijkstras import BiDirectionalDijkstra
from modules.path_planning.RRT_star import RRT_Star
from modules.path_planning.INFORMED_RRT import Informed_RRT_star
from modules.path_planning.RRT import RRTtree

import time
import random
import math
import os
import copy
from os.path import isfile,join

#Global variables to store the mouse position coordinates from the image
startX, startY, endX, endY = 0,0,0,0
selectStart = True
img = None

def select_point(event,x,y,flags,param):
    """
    Grabs points selected from the cv2 image shown on screen
    """
    global startX,startY,endX,endY,img,selectStart
    

        

    if event == cv2.EVENT_LBUTTONDBLCLK: # captures left button double-click
        ix,iy = x,y

        #Check to see if the user selected a point inside a road
        if not np.array_equal(img[iy][ix],[255,255,255]):
            print("Please select a valid point inside the road!")
            return
            
        #Set the start coordinate if selectStart is still true
        if selectStart:
            img = cv2.circle(img,(ix,iy),radius=5,color=(0,255,0),thickness=-1)
            selectStart = False
            startX,startY = ix,iy
            cv2.imshow('image',img)
            cv2.waitKey(1)
        #Set the endPoint and begin path planning computation
        else:
            img = cv2.circle(img,(ix,iy),radius=5,color=(0,0,255),thickness=-1)
            endX,endY = ix,iy
            cv2.imshow('image',img)
            cv2.waitKey(1)

            #Start the path planning algoirthm
            runPathPlanning()

            #Display the results to the user
            #Resize the image if needed
            #img = cv2.resize(img,(1024,1024),interpolation=cv2.INTER_AREA)
            cv2.imshow('image',img)
            selectStart = True


def grabRandomImage():
    #Returns an image path to use for the path processing

    imgFolder = "C:/Users/charles/Downloads/Path-Planning-On-Aerial-Images-main-20220523T022800Z-001/Path-Planning-On-Aerial-Images-main/data/processedImages/"
    imgs = [f for f in os.listdir(imgFolder) if isfile(join(imgFolder,f))]

    #return join(imgFolder,random.choice(imgs))
    return join(imgFolder, imgs[3])

def runPathPlanning(algoName="INFORMED RRT*"):
    
    start = time.time()
    startPos = (startX,startY)
    endPos = (endX,endY)

    if algoName == "astar":

        #Run the pathfinding algorithm
        aStarAlgo = AStar(startPos,endPos,copy.deepcopy(img),"image")
        path = aStarAlgo.run()
        print("Path length:",len(path))
    
    elif algoName == "bfs":
        bfsAlgo = BFS(startPos,endPos,copy.deepcopy(img),"image")
        path = bfsAlgo.run()
    
    elif algoName == "greedyBestFirst":
        greedyBestFirstAlgo = greedyBestFirst(startPos,endPos,copy.deepcopy(img),"image")
        path = greedyBestFirstAlgo.run()
    elif algoName == "bidirectDijkstras":
        bidirectDijk = BiDirectionalDijkstra(startPos,endPos,copy.deepcopy(img),"image")
        path = bidirectDijk.run()
    elif algoName == "RRT":
        RRT = RRTtree(startPos, endPos, 4, 10000, copy.deepcopy(img), "image")
        path = RRT.run()
    
    elif algoName == "RRT_Star":
        RRT_STAR = RRT_Star(startPos, endPos, 10, 10000, 50, copy.deepcopy(img))
        path = RRT_STAR.run()
    
    elif algoName == "INFORMED RRT*":
        Informed_RRT = Informed_RRT_star(startPos, endPos, 10000, 10, 30, 150, copy.deepcopy(img))
        path = Informed_RRT.run()

    else:
        print("No valid algorithm found for the algorithm name given...exiting")
        exit()
    #else algoName == "LPA":
    #Future path finding algorithms go here..each should return a list of coordinates
    

    end = time.time()
    duration = end - start
    #cv2.line(img, start point, end point, color=(0, 255, 0), 3)
    #Plot the path found onto the image
    solution_quality = 0.0
    map_path = []
    first_point = path[0]
    second_point = None

    for pos in path:
        map_path.append(pos)
  #      cv2.rectangle(img,pos,(pos[0]+2,pos[1]+2),color=(0,0,255),thickness=-1)
        if pos != first_point:
            second_point = pos
            cv2.line(img, first_point, second_point, color=(0, 255, 0), thickness=3)
            first_point = pos
        if pos != path[0]:
            solution_quality += math.hypot((map_path[-1][0] - map_path[-2][0]),(map_path[-1][1] - map_path[-2][1]))
        #img[pos[1]][pos[0]] = [0,255,0]

    print('Quality/Cost of solution: ', solution_quality)
    print("Time took: ",duration)


def main():
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    global img 
    imgPath = grabRandomImage()

    #Load the image
    img = cv2.imread(imgPath)

    #Resize the image if desired (Speeds up computation)
    #img = cv2.resize(img,(512,512),interpolation=cv2.INTER_AREA)
    #Convert the image to GrayScale
    #img = cv2.cvtColor(cv2.imread('path1.png'),cv2.COLOR_BGR2GRAY)

    #Initialize the image window
    cv2.namedWindow('image')
    cv2.setMouseCallback('image',select_point)


    #Display image and wait until the user selects two valid points on the image 
    cv2.imshow('image',img)
    cv2.waitKey(0) #The bulk of the program logic is run through the select_point() function that processes the mouse clicks from the user

    cv2.destroyAllWindows()



if __name__ == "__main__":
    main()

Beginning Informed RRT* Search from  (5, 125)  to  (1018, 770)
